<a href="https://colab.research.google.com/github/SewoongPark/SeSac_study_repo/blob/main/study_00.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **TIL**
### 프로젝트
* 프로젝트(포트폴리오)에서 보여줘야 될 것은 여러가지를 알고 있다는 것
* 좋은 성능을 강조하기 보다 최대한 많은 기술을 담아야 함
* 여러 모델들의 특징을 간략하게 설명한 뒤에 본인의 데이터셋으로 학습을 한 결과를 보여주고, baseline 모델 선택에 대한 이유를 제시해야 함

**keras tuner 사용해보기**
> <a>https://www.tensorflow.org/tutorials/keras/keras_tuner?hl=ko</a>

* 전이학습에서의 fine tunning
  * 학습된 모델의 레이어와 weight를 어디까지 사용할지를 지정해주는 것
  
> 전이 학습(Transfer Learning)에서 fine-tuning은 이미 학습된 모델의 일부 레이어나 가중치를 새로운 작업에 맞게 조정하는 과정을 말합니다.<br> 기존에 풀려고 했던 작업과 유사한 작업을 수행하는 새로운 작업에 대해 기존 모델의 일부를 재사용하여 성능을 향상시키는 기법입니다.

* Fine-tuning은 일반적으로 다음과 같은 단계로 이루어집니다:

  * Pre-trained Model 선택: 특정 작업에 가장 적합한 사전 학습된 모델을 선택합니다. 이는 대규모 이미지 데이터셋 (예: ImageNet)에서 학습된 VGG, ResNet, 또는 텍스트 데이터에서 학습된 BERT 등과 같은 모델일 수 있습니다.
  * 모델의 일부 레이어 고정: 주로 모델의 초기 레이어들을 고정하고, 특정 레이어 이후의 레이어들을 새로운 데이터셋에 맞게 fine-tuning합니다. 고정된 레이어들은 이전에 학습된 특징 추출기로서의 역할을 수행하며, 새로운 데이터셋에 맞게 그대로 사용됩니다.
  * 학습률 조정: 새로운 데이터셋에서의 fine-tuning을 위해 학습률(learning rate)을 조정합니다. 보통 이전 레이어들은 낮은 학습률로 설정하고, fine-tuning할 레이어들은 상대적으로 높은 학습률을 사용합니다.
  * 새로운 데이터셋 학습: 주어진 새로운 작업에 대해 fine-tuning을 진행합니다. 이는 새로운 데이터셋에 대해 모델을 재조정하는 과정으로, 모델이 새로운 작업에 더 잘 맞도록 하는 것입니다.
  * 성능 평가: Fine-tuning된 모델을 평가하여 새로운 작업에 대한 성능을 평가합니다. 이는 정확도, 정밀도, 재현율 등의 지표를 사용하여 이루어질 수 있습니다.

**최대한 많은 기술을 사용했다는 것을 보여주기:** <br>
  * depthwise separable convolution, xception, inception, VGG, ResNet50 모델 등 사용해보기
* 참고자료: <a>https://underflow101.tistory.com/42

### 평가: 전이학습을 이용하여서 본인만의 분류학습 모델을 제작 및 파인튜닝 과정을 pt로 정리

## 1. 데이터셋 준비

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [5]:
!unzip -qq "/content/archive.zip"

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import optimizers

import numpy as np
import matplotlib.pyplot as plt

import itertools
import imutils
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
from plotly import tools

from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16, preprocess_input
from keras import layers
from keras.models import Model, Sequential
from keras.optimizers import Adam, RMSprop
from keras.callbacks import EarlyStopping

init_notebook_mode(connected=True)
RANDOM_SEED = 123

In [26]:
# 학습셋의 변형을 설정하는 부분입니다.
train_datagen = ImageDataGenerator(rescale=1./255,          # 주어진 이미지의 크기를 설정합니다.
                                  # horizontal_flip=True,     # 수평 대칭 이미지를 50% 확률로 만들어 추가합니다.
                                  # width_shift_range=0.1,    # 전체 크기의 15% 범위에서 좌우로 이동합니다.
                                  # height_shift_range=0.1,   # 마찬가지로 위, 아래로 이동합니다.
                                  #rotation_range=5,        # 정해진 각도만큼 회전시킵니다.
                                  #shear_range=0.7,         # 좌표 하나를 고정시키고 나머지를 이동시킵니다.
                                  #zoom_range=1.2,          # 확대 또는 축소시킵니다.
                                  #vertical_flip=True,      # 수직 대칭 이미지를 만듭니다.
                                  #fill_mode='nearest'      # 빈 공간을 채우는 방법입니다. nearest 옵션은 가장 비슷한 색으로 채우게 됩니다.
                                  )

train_generator = train_datagen.flow_from_directory(
       '/content/TRAIN',   # 학습셋이 있는 폴더의 위치입니다.
       target_size=(150, 150),
       batch_size=5,
       class_mode='binary')

# 테스트셋은 이미지 부풀리기 과정을 진행하지 않습니다.
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
       '/content/TEST',   # 테스트셋이 있는 폴더의 위치입니다.
       target_size=(150, 150),
       batch_size=5,
       class_mode='binary')

Found 193 images belonging to 2 classes.
Found 10 images belonging to 2 classes.


In [33]:
fig, axes = plt.subplots(1, num_images, figsize=(20, 20))

    for i, (image, label) in enumerate(zip(images, labels)):
        axes[i].imshow(image)
        axes[i].set_title(class_names[label])
        axes[i].axis('off')

    plt.show()

KeyboardInterrupt: 

In [ ]:
!apt-get install tree
#clear_output()
# create new folders
!mkdir TRAIN TEST VAL TRAIN/YES TRAIN/NO TEST/YES TEST/NO VAL/YES VAL/NO
!tree -d

In [12]:
import os
import shutil

IMG_PATH = '/content/brain_tumor_dataset/'
# split the data by train/val/test
for CLASS in os.listdir(IMG_PATH):
    if not CLASS.startswith('.'):
        IMG_NUM = len(os.listdir(IMG_PATH + CLASS))
        for (n, FILE_NAME) in enumerate(os.listdir(IMG_PATH + CLASS)):
            img = IMG_PATH + CLASS + '/' + FILE_NAME
            if n < 5:
                shutil.copy(img, 'TEST/' + CLASS.upper() + '/' + FILE_NAME)
            elif n < 0.8*IMG_NUM:
                shutil.copy(img, 'TRAIN/'+ CLASS.upper() + '/' + FILE_NAME)
            else:
                shutil.copy(img, 'VAL/'+ CLASS.upper() + '/' + FILE_NAME)

In [24]:
import tqdm as tq
import cv2

def load_data(dir_path, img_size=(100,100)):
    """
    Load resized images as np.arrays to workspace
    """
    X = []
    y = []
    i = 0
    labels = dict()
    for path in tq.tqdm(sorted(os.listdir(dir_path))):
        if not path.startswith('.'):
            labels[i] = path
            for file in os.listdir(dir_path + path):
                if not file.startswith('.'):
                    img = cv2.imread(dir_path + path + '/' + file)
                    X.append(img)
                    y.append(i)
            i += 1
    X = np.array(X)
    y = np.array(y)
    print(f'{len(X)} images loaded from {dir_path} directory.')
    return X, y, labels



def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.figure(figsize = (6,6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    cm = np.round(cm,2)
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

TRAIN_DIR = 'TRAIN/'
TEST_DIR = 'TEST/'
VAL_DIR = 'VAL/'
IMG_SIZE = (224,224)

# use predefined function to load the image data into workspace
X_train, y_train, labels = load_data(TRAIN_DIR, IMG_SIZE)
X_test, y_test, _ = load_data(TEST_DIR, IMG_SIZE)
X_val, y_val, _ = load_data(VAL_DIR, IMG_SIZE)

100%|██████████| 2/2 [00:00<00:00,  3.62it/s]


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (193,) + inhomogeneous part.

TypeError: 'module' object is not callable